<a href="https://colab.research.google.com/github/mitalidb/projects/blob/master/BotChronicKidney.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install newspaper3k

     |████████████████████████████████| 215kB 2.8MB/s 
     |████████████████████████████████| 7.4MB 8.6MB/s 
     |████████████████████████████████| 194kB 43.9MB/s 
     |████████████████████████████████| 51kB 7.3MB/s 
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-cp36-none-any.whl size=13539 sha256=784f2963230d99d233dc2624973452c3ab9034c7f34cec9e113e2ac560cdbc6a
  Stored in directory: /root/.cache/pip/wheels/81/2b/43/a02ede72324dd40cdd7ca53aad718c7710628e91b8b0dc0f02
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-cp36-none-any.whl size=3357 sha256=84997dc8b1b78c54194753ba600de8ef007b6fa0bcc87bb0893313393c2e2fac
  Stored in directory: /root/.cache/pip/wheels/de/03/ca/778e3a7a627e3d98836cc890e7cb40c7575424cfd3340f40ed
  Created wheel for jieba3k: filename=jieba3k-0.35.1-cp36-none-any.whl size=7398406 sha256=728395b2fc133129fa161935d2a48bb59f2b66f2787101990df688c0fa54be1f
  Stored in directory: /root/.cache/pip/wheels/83/15/9c/a3f1f67e7f7181170ad37d32e503c35

In [0]:
import pandas as pd
import numpy as np
import re
import string
import nltk
import warnings
from newspaper import Article
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [0]:
warnings.filterwarnings('ignore')

In [4]:
nltk.download('wordnet',quiet= True)
nltk.download('punkt', quiet= True)

True

In [5]:
#Get the Articcle URL
article=Article('https://www.mayoclinic.org/diseases-conditions/chronic-kidney-disease/symptoms-causes/syc-20354521')
article.download()
article.parse()
article.nlp()
corpus= article.text

#Print the corpus
print(corpus)

Overview

Chronic kidney disease, also called chronic kidney failure, describes the gradual loss of kidney function. Your kidneys filter wastes and excess fluids from your blood, which are then excreted in your urine. When chronic kidney disease reaches an advanced stage, dangerous levels of fluid, electrolytes and wastes can build up in your body.

In the early stages of chronic kidney disease, you may have few signs or symptoms. Chronic kidney disease may not become apparent until your kidney function is significantly impaired.

Treatment for chronic kidney disease focuses on slowing the progression of the kidney damage, usually by controlling the underlying cause. Chronic kidney disease can progress to end-stage kidney failure, which is fatal without artificial filtering (dialysis) or a kidney transplant.

Chronic kidney disease care at Mayo Clinic

How kidneys work

Symptoms

Signs and symptoms of chronic kidney disease develop over time if kidney damage progresses slowly. Signs an

In [6]:
#Tokenization
text= corpus
sent_token= nltk.sent_tokenize(text) #convert text into  list of sentence
#print the sentence
print(sent_token) 

['Overview\n\nChronic kidney disease, also called chronic kidney failure, describes the gradual loss of kidney function.', 'Your kidneys filter wastes and excess fluids from your blood, which are then excreted in your urine.', 'When chronic kidney disease reaches an advanced stage, dangerous levels of fluid, electrolytes and wastes can build up in your body.', 'In the early stages of chronic kidney disease, you may have few signs or symptoms.', 'Chronic kidney disease may not become apparent until your kidney function is significantly impaired.', 'Treatment for chronic kidney disease focuses on slowing the progression of the kidney damage, usually by controlling the underlying cause.', 'Chronic kidney disease can progress to end-stage kidney failure, which is fatal without artificial filtering (dialysis) or a kidney transplant.', 'Chronic kidney disease care at Mayo Clinic\n\nHow kidneys work\n\nSymptoms\n\nSigns and symptoms of chronic kidney disease develop over time if kidney damage

In [7]:
#create a dict(key value pair) to remove punct
remove_punct_dict= dict(   (ord(punct),None) for punct in string.punctuation)

print(string.punctuation)
#print the dic
print(remove_punct_dict)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
{33: None, 34: None, 35: None, 36: None, 37: None, 38: None, 39: None, 40: None, 41: None, 42: None, 43: None, 44: None, 45: None, 46: None, 47: None, 58: None, 59: None, 60: None, 61: None, 62: None, 63: None, 64: None, 91: None, 92: None, 93: None, 94: None, 95: None, 96: None, 123: None, 124: None, 125: None, 126: None}


In [8]:
#create a function to return a list of lemmatized lower case after removing punctuation
def lemNormalize(text):
  return nltk.word_tokenize(text.lower().translate(remove_punct_dict))

print(lemNormalize(text))

['overview', 'chronic', 'kidney', 'disease', 'also', 'called', 'chronic', 'kidney', 'failure', 'describes', 'the', 'gradual', 'loss', 'of', 'kidney', 'function', 'your', 'kidneys', 'filter', 'wastes', 'and', 'excess', 'fluids', 'from', 'your', 'blood', 'which', 'are', 'then', 'excreted', 'in', 'your', 'urine', 'when', 'chronic', 'kidney', 'disease', 'reaches', 'an', 'advanced', 'stage', 'dangerous', 'levels', 'of', 'fluid', 'electrolytes', 'and', 'wastes', 'can', 'build', 'up', 'in', 'your', 'body', 'in', 'the', 'early', 'stages', 'of', 'chronic', 'kidney', 'disease', 'you', 'may', 'have', 'few', 'signs', 'or', 'symptoms', 'chronic', 'kidney', 'disease', 'may', 'not', 'become', 'apparent', 'until', 'your', 'kidney', 'function', 'is', 'significantly', 'impaired', 'treatment', 'for', 'chronic', 'kidney', 'disease', 'focuses', 'on', 'slowing', 'the', 'progression', 'of', 'the', 'kidney', 'damage', 'usually', 'by', 'controlling', 'the', 'underlying', 'cause', 'chronic', 'kidney', 'disease'

In [0]:
#keyword matching
#greeting input
Greeting_input= ['hi','hello', 'hola', 'greeting','wassup', 'hey','hie']
#Greeting response  back to the user
Greeting_responces=['howdy', 'hi','hie', "what's up", 'hey there','Hello']
#Function to return a random greeting response
def greeting(sentence):
  #if an user input is  greeting we want to return a randomly choosen greeting responce
  for word in sentence.split():
    if word.lower() in Greeting_input:
      return random.choice(Greeting_responces)



In [10]:
#Generate response
def response(user_response):
  #The users response or greeting
  #user_response= 'What is chronic kidney disease'
  user_response= user_response.lower() # making the response lower 
  #print(user_response)
  #set the chatbpot response 
  robo_response= ''

  #Append the user resoponse to the sentense list 
  sent_token.append(user_response)

  #print(sent_token)
  #Create a TfidfVectorizer object
  TfidfVec = TfidfVectorizer(tokenizer=lemNormalize, stop_words='english')

  #convert the text to matrix of TF-IDF feature 
  tfidf= TfidfVec.fit_transform(sent_token)
  #print(tfidf)

  #gwet the measure of similarity score
  vals= cosine_similarity(tfidf[-1], tfidf)
  #print(vals)
  #get the index of  the most similar/sentense to the user response
  idx= vals.argsort()[0][-2]

  #reduce the  dimentionality of the vals
  flat= vals.flatten()

  #sort the lsit in ascending order
  flat.sort()

  #Get the most similar score to the user responses
  score= flat[-2]

  #If the ariable score is zero then there is no text similar to user response 
  if(score== 0):
    robo_response= robo_response+"I apolosize, I dont understand."
  else:
    robo_response= robo_response+sent_token[idx]

  #print(robo_response)
  sent_token.remove(user_response)
  return robo_response

print(robo_response)

NameError: ignored

In [0]:
flag= True
print("DOCBot: I am a doctor bot or DOCBot for short.I will answer all your question abput the chromnic kidney Disease.If you want to exit type BYe!")
while(flag== True):
  user_response=input()
  user_response= user_response.lower()
  if(user_response != 'bye'):
    if(user_response== 'thanks' or user_response== 'thank you'):
      flag= False
      print('DOCBot: You are welcome!')
    else:
      if(greeting(user_response)!= None):
        print('DOCBot:'+greeting(user_response))
      else:
        print("DOCBot: "+response(user_response))
  else:
    flag= False
    print('DOCBot: Chat with you later.')
